In [ ]:
!pip install 'aif360[LFR]'

     |████████████████████████████████| 175 kB 8.4 MB/s 
     |████████████████████████████████| 564 kB 55.0 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=8549bfe1b2528a524c45b968e568bcbd2568e8945f04718739c7df328b1bf811
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
!pip install fairlearn==0.4.6

     |████████████████████████████████| 21.2 MB 1.4 MB/s 


In [ ]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test


--2022-05-20 09:17:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  --.-KB/s    in 0.1s    

2022-05-20 09:17:58 (31.0 MB/s) - ‘adult.data’ saved [3974305/3974305]

--2022-05-20 09:17:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2022-

In [ ]:
#Code written by V Sharmanska. All rights reserved.
#STEP 1: Import the AIF360 library and set the random seed.
import numpy as np
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult,load_preproc_data_german
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import LFR, Reweighing
from aif360.sklearn.inprocessing import ExponentiatedGradientReduction 
from sklearn.preprocessing import StandardScaler  #MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import *
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import GermanDataset
np.random.seed(0)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from numpy.random import randn
from sklearn.preprocessing import StandardScaler
from tqdm import notebook
import torch
import torch.nn as nn
from torch.nn import functional as F
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
import torch.optim as optim
from tqdm import tqdm


In [ ]:
#STEP 2: We define where's the bias in the features of our dataset.

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])


#STEP 3: We split between training and test set.
train, test = dataset_orig.split([0.7], shuffle=True)
print("training data size", train.features.shape)
print("dataset feature names", train.feature_names)


#Normalize the dataset, both train and test. This should always be done in any machine learning pipeline!
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(train.features)
y_train = train.labels.ravel()

X_test = scale_orig.transform(test.features) 
y_test = test.labels.ravel()

training data size (34189, 18)
dataset feature names ['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


In [ ]:
from torch.utils.data.dataset import Subset


In [ ]:
#q1 
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
Cs=[0,0.001,0.003]
n_estimat=[10,100]
criter=["gini","entropy"]
listdf=[]
listdf_test=[]
k=0
for t,v in kf.split(X_train):
  k+=1
  trainx, valx = X_train[t],X_train[v]
  trainy,valy = y_train[t],y_train[v]
  trainsb = train.subset(indexes=t)
  for crite in criter: 
    for n_esti in n_estimat:
      for c in Cs:
        print("k= ",k)
        print("number of trees= ",n_esti)
        print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
        print("alpha = ",c)
        classifier = RandomForestClassifier(n_estimators = n_esti, criterion = crite, random_state = 0,ccp_alpha=c)
        classifier.fit(trainx,trainy)
        predictions = classifier.predict(trainx)

        test_pred = trainsb.copy()
        predictions.resize((len(predictions),1))
        test_pred.labels = predictions

        #print("Accuracy", sum(y_pred==y_test)/len(y_test))
        metric = ClassificationMetric(trainsb, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs = {}
        metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
        metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
        metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
        metric_arrs['bal_acc']=((metric.true_positive_rate() + metric.true_negative_rate()) / 2)
        print(metric_arrs)
        cm = confusion_matrix(trainy, predictions)
        print(cm," cm")
        print(metric.accuracy()," !accuracy!")
        a=metric.accuracy()
        listdf.append([k,crite,n_esti,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc']])

        #the values get when the current classifier applied to the Test set
        predicted_test_values=classifier.predict(X_test)
        test_pred_for_test = test.copy()
        predicted_test_values.resize((len(predicted_test_values),1))
        test_pred_for_test.labels = predicted_test_values
        metric_test = ClassificationMetric(test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs_tets = {}
        metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
        metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
        metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
        metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
        a_test=metric_test.accuracy()
        listdf_test.append([k,crite,n_esti,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc']])

    




k=  1
number of trees=  10
6838-34188 0-6837
alpha =  0
{'stat_par_diff': -0.2274469016860083, 'eq_opp_diff': -0.4856424982053123, 'avg_odds_diff': -0.2998864664939605, 'bal_acc': 0.6710064693516796}
[[19330  1449]
 [ 3866  2706]]  cm
0.8056743811926438  !accuracy!
k=  1
number of trees=  10
6838-34188 0-6837
alpha =  0.001
{'stat_par_diff': -0.22241077293628203, 'eq_opp_diff': -0.47361809045226133, 'avg_odds_diff': -0.29288970055064234, 'bal_acc': 0.666510655407074}
[[19355  1424]
 [ 3933  2639]]  cm
0.804138788344119  !accuracy!
k=  1
number of trees=  10
6838-34188 0-6837
alpha =  0.003
{'stat_par_diff': -0.22241077293628203, 'eq_opp_diff': -0.47361809045226133, 'avg_odds_diff': -0.29288970055064234, 'bal_acc': 0.666510655407074}
[[19355  1424]
 [ 3933  2639]]  cm
0.804138788344119  !accuracy!
k=  1
number of trees=  100
6838-34188 0-6837
alpha =  0
{'stat_par_diff': -0.22065907597985548, 'eq_opp_diff': -0.4748743718592965, 'avg_odds_diff': -0.2919819244299633, 'bal_acc': 0.66798166

In [ ]:
import pandas as pd
df = pd.DataFrame (listdf, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc'])
df

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc
0,1,gini,10,0.000,80.567438,-0.227447,-0.485642,-0.299886,0.671006
1,1,gini,10,0.001,80.413879,-0.222411,-0.473618,-0.292890,0.666511
2,1,gini,10,0.003,80.413879,-0.222411,-0.473618,-0.292890,0.666511
3,1,gini,100,0.000,80.582063,-0.220659,-0.474874,-0.291982,0.667982
4,1,gini,100,0.001,80.413879,-0.222411,-0.473618,-0.292890,0.666511
5,1,gini,100,0.003,80.413879,-0.222411,-0.473618,-0.292890,0.666511
6,1,entropy,10,0.000,80.567438,-0.227447,-0.485642,-0.299886,0.671006
7,1,entropy,10,0.001,80.413879,-0.222411,-0.473618,-0.292890,0.666511
8,1,entropy,10,0.003,80.413879,-0.222411,-0.473618,-0.292890,0.666511
9,1,entropy,100,0.000,80.582063,-0.220659,-0.474874,-0.291982,0.667982


In [ ]:
import pandas as pd
df_test = pd.DataFrame (listdf_test, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc'])
df_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc
0,1,gini,10,0.000,80.379444,-0.222796,-0.469024,-0.292918,0.665173
1,1,gini,10,0.001,80.434041,-0.214488,-0.456566,-0.283428,0.661840
2,1,gini,10,0.003,80.434041,-0.214488,-0.456566,-0.283428,0.661840
3,1,gini,100,0.000,80.420392,-0.213475,-0.454545,-0.282128,0.661152
4,1,gini,100,0.001,80.434041,-0.214488,-0.456566,-0.283428,0.661840
5,1,gini,100,0.003,80.434041,-0.214488,-0.456566,-0.283428,0.661840
6,1,entropy,10,0.000,80.379444,-0.222796,-0.469024,-0.292918,0.665173
7,1,entropy,10,0.001,80.434041,-0.214488,-0.456566,-0.283428,0.661840
8,1,entropy,10,0.003,80.434041,-0.214488,-0.456566,-0.283428,0.661840
9,1,entropy,100,0.000,80.420392,-0.213475,-0.454545,-0.282128,0.661152


In [ ]:
#best fairness
column = df["eq_opp_diff"].abs()
max_index_bf = column.idxmin()
print("best value for Equal opportunity difference is %s and its index is %s" % (round(df["eq_opp_diff"][max_index_bf],6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % df['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % df['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is -0.449177 and its index is 31
best Equal opportunity difference's fold is 3
cpp_alpha value for the best Equal opportunity difference is 0.001 


In [ ]:
#best accuracy
column = df["accuracy"]
max_value_ba = column.max()
max_index_ba = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_ba,6),max_index_ba))
print("best accuracy's fold is %s" % df['kfold'][max_index_ba])
print("cpp_alpha value for the best accuracy is %s " % df['cpp_alpha'][max_index_ba])


maximum value for accuracy is 80.582063 and its index is 3
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0 


In [ ]:
#best fairness and accuracy models are applied to test set and their values are shown
columna = df_test["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  df_test["eq_opp_diff"].abs()
max_indexf = columnf.idxmin()


print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (df_test["accuracy"][max_index_bf],df_test["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (df_test["accuracy"][max_index_ba],df_test["eq_opp_diff"][max_index_ba]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,df_test['kfold'][max_indexa],df_test['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(df_test["eq_opp_diff"][max_indexf],df_test['kfold'][max_indexf],df_test['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 80.45451443390432 and its Equal opportunity difference is -0.4292929292929293


 the accuracy of the test set when the best accuracy model is applied to, is 80.42039172865626 and its Equal opportunity difference is -0.45454545454545453


 the best accuracy value revealed for the test set is 80.4886371391524 and get from fold 2 with cpp_alpha value 0.001


 the best Equal opportunity difference value revealed for the test set is -0.42828282828282827 and get from fold 2 with cpp_alpha value 0.001


In [ ]:
#RW q2
from sklearn.model_selection import KFold
from scipy.special import logsumexp
kf = KFold(n_splits=5)
Cs=[0,0.001,0.003]
n_estimat=[10,100]
criter=["gini","entropy"]
RW_model = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
k=0
RWlistdf=[]
listdf_test_RW=[]
for t,v in kf.split(X_train):
  k+=1
  trainx, valx = X_train[t],X_train[v]
  trainy,valy = y_train[t],y_train[v]
  trainsb = train.subset(indexes=t)
  for crite in criter:
    for n_estim in n_estimat:
      for c in Cs:
        trainsb = RW_model.fit_transform(trainsb)
        learner = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
        learner.fit(trainx,trainy,sample_weight=trainsb.instance_weights)

        predictions = learner.predict(trainx)
      
        test_pred = trainsb.copy()
        predictions.resize((len(predictions),1))
        test_pred.labels = predictions

        metric = ClassificationMetric(trainsb, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs = {}
        metric_arrs['stat_par_diff']=(logsumexp(metric.statistical_parity_difference()))
        metric_arrs['eq_opp_diff']=(logsumexp(metric.equal_opportunity_difference()))
        metric_arrs['avg_odds_diff']=(logsumexp(metric.average_odds_difference()))
        metric_arrs['bal_acc']=((logsumexp(metric.true_positive_rate())+ logsumexp(metric.true_negative_rate())) / 2)
        metric_arrs['disp_imp']=(logsumexp(metric.disparate_impact()))
        print(metric_arrs)

        print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
        print("alpha = ",c)
        a=metric.accuracy()
        print("! accuracy !",a*100)
        RWlistdf.append([k,crite,n_estim,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc'],metric_arrs['disp_imp']])

        #the values get when the current classifier applied to the Test set
        predicted_test_values=learner.predict(X_test)
        test_pred_for_test = test.copy()
        predicted_test_values.resize((len(predicted_test_values),1))
        test_pred_for_test.labels = predicted_test_values
        metric_test = ClassificationMetric(test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs_tets = {}
        metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
        metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
        metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
        metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
        a_test=metric_test.accuracy()
        listdf_test_RW.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc'],metric_arrs['disp_imp']])




    



  

{'stat_par_diff': -0.03240975130600171, 'eq_opp_diff': -0.07164465183058144, 'avg_odds_diff': -0.04582257751882966, 'bal_acc': 0.6712502178040349, 'disp_imp': 0.8306806426313541}
6838-34188 0-6837
alpha =  0
! accuracy ! 79.09264698640082
{'stat_par_diff': -0.003246486764995754, 'eq_opp_diff': -0.004263460157932553, 'avg_odds_diff': -0.0035941488515604247, 'bal_acc': 0.6659450600947081, 'disp_imp': 0.9818412298551537}
6838-34188 0-6837
alpha =  0.001
! accuracy ! 78.85817651780783
{'stat_par_diff': 0.010060715028679529, 'eq_opp_diff': 0.023736539842067528, 'avg_odds_diff': 0.014735926615469193, 'bal_acc': 0.6691563475991127, 'disp_imp': 1.0562732039041076}
6838-34188 0-6837
alpha =  0.003
! accuracy ! 78.86311425039635
{'stat_par_diff': -0.033508907058093895, 'eq_opp_diff': -0.07397774587221834, 'avg_odds_diff': -0.04734356688369347, 'bal_acc': 0.671792630448373, 'disp_imp': 0.8259378132730215}
6838-34188 0-6837
alpha =  0
! accuracy ! 79.09411994613367
{'stat_par_diff': 0.000712808414

In [ ]:
import pandas as pd
dfRW = pd.DataFrame (RWlistdf, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfRW

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.000,79.092647,-0.032410,-0.071645,-0.045823,0.671250,0.830681
1,1,gini,10,0.001,78.858177,-0.003246,-0.004263,-0.003594,0.665945,0.981841
2,1,gini,10,0.003,78.863114,0.010061,0.023737,0.014736,0.669156,1.056273
3,1,gini,100,0.000,79.094120,-0.033509,-0.073978,-0.047344,0.671793,0.825938
4,1,gini,100,0.001,79.001385,0.000713,-0.005492,-0.001409,0.672638,1.003871
5,1,gini,100,0.003,78.991456,0.005649,0.010094,0.007168,0.663544,1.033250
6,1,entropy,10,0.000,79.092647,-0.032410,-0.071645,-0.045823,0.671250,0.830681
7,1,entropy,10,0.001,78.840740,0.009542,0.023351,0.014263,0.666020,1.054531
8,1,entropy,10,0.003,78.979011,0.000194,-0.005878,-0.001882,0.669502,1.001075
9,1,entropy,100,0.000,79.094120,-0.033509,-0.073978,-0.047344,0.671793,0.825938


In [ ]:
import pandas as pd
df_test_RW = pd.DataFrame (listdf_test_RW, columns =['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
df_test_RW

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.000,79.164676,-0.088332,-0.051831,-0.034784,0.669090,0.830681
1,1,gini,10,0.001,78.905344,-0.060279,0.026579,0.013297,0.661009,0.981841
2,1,gini,10,0.003,78.700607,-0.050244,0.044871,0.026988,0.660566,1.056273
3,1,gini,100,0.000,79.151027,-0.089142,-0.052841,-0.035652,0.669300,0.825938
4,1,gini,100,0.001,79.055484,-0.057724,0.035132,0.019936,0.667178,1.003871
5,1,gini,100,0.003,78.946291,-0.048932,0.044424,0.026495,0.655990,1.033250
6,1,entropy,10,0.000,79.164676,-0.088332,-0.051831,-0.034784,0.669090,0.830681
7,1,entropy,10,0.001,78.680134,-0.048922,0.043463,0.026423,0.657340,1.054531
8,1,entropy,10,0.003,79.035010,-0.056402,0.033723,0.019371,0.663952,1.001075
9,1,entropy,100,0.000,79.151027,-0.089142,-0.052841,-0.035652,0.669300,0.825938


In [ ]:
#best fairness
column = dfRW["eq_opp_diff"].abs()
max_value_bf = column.min()
max_index_bf = column.idxmin()
print("best value for Equal opportunity difference is %s and its index is %s" % (round(dfRW["eq_opp_diff"][max_index_bf],6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % df['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % df['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is -0.000537 and its index is 26
best Equal opportunity difference's fold is 3
cpp_alpha value for the best Equal opportunity difference is 0.003 


In [ ]:
#best accuracy
column = dfRW["accuracy"]
max_value_a = column.max()
max_index_a = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_a,6),max_index_a))
print("best accuracy's fold is %s" % df['kfold'][max_index_a])
print("cpp_alpha value for the best accuracy is %s " % df['cpp_alpha'][max_index_a])


maximum value for accuracy is 79.09412 and its index is 3
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0 


In [ ]:
#best fairness adn accuracy models are applied to test set and their values are shown
columna = df_test_RW["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  df_test_RW["eq_opp_diff"].abs()
max_indexf = columnf.idxmin()
max_valuef = df_test_RW["eq_opp_diff"][max_indexf]



print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (df_test_RW["accuracy"][max_index_bf],df_test_RW["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (df_test_RW["accuracy"][max_index_a],df_test_RW["eq_opp_diff"][max_index_a]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,df_test_RW['kfold'][max_indexa],df_test_RW['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuef,df_test_RW['kfold'][max_indexf],df_test_RW['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 79.03500989558452 and its Equal opportunity difference is 0.03372341299170567


 the accuracy of the test set when the best accuracy model is applied to, is 79.05548351873335 and its Equal opportunity difference is 0.03513180586351322


 the best accuracy value revealed for the test set is 79.58779772060329 and get from fold 5 with cpp_alpha value 0.003


 the best Equal opportunity difference value revealed for the test set is 0.0023363718485669427 and get from fold 4 with cpp_alpha value 0.0


In [ ]:
#EGR q2
from sklearn.model_selection import KFold
from scipy.special import logsumexp
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction
kf = KFold(n_splits=5)
Cs=[0,0.001,0.003]
n_estimat=[10,100]
criter=["gini","entropy"]
EGRlistdf=[]
EGRlistdf_test=[]
k=0
for t,v in kf.split(X_train):
  k+=1
  trainx, valx = X_train[t],X_train[v]
  trainy,valy = y_train[t],y_train[v]
  trainsb = train.subset(indexes=t)
  for crite in criter:
    for n_estim in n_estimat:
      for c in Cs:
        learner = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
        exp_grad_red = ExponentiatedGradientReduction(estimator=learner, 
                                                  constraints="EqualizedOdds",
                                                  drop_prot_attr=False)
        exp_grad_red.fit(trainsb)

        exp_grad_red_pred = exp_grad_red.predict(trainsb)

        metric = ClassificationMetric(trainsb, 
                                      exp_grad_red_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
        
      
        metric_arrs = {}
        metric_arrs['stat_par_diff']=(logsumexp(metric.statistical_parity_difference()))
        metric_arrs['eq_opp_diff']=(logsumexp(metric.equal_opportunity_difference()))
        metric_arrs['avg_odds_diff']=(logsumexp(metric.average_odds_difference()))
        metric_arrs['bal_acc']=((logsumexp(metric.true_positive_rate())+ logsumexp(metric.true_negative_rate())) / 2)
        metric_arrs['disp_imp']=(logsumexp(metric.disparate_impact()))
        print(metric_arrs)
        print(metric.accuracy()," !accuracy!")

        print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
        print("alpha = ",c)
        a=metric.accuracy()
        EGRlistdf.append([k,crite,n_estim,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc'],metric_arrs['disp_imp']])

        #the values get when the current classifier applied to the Test set
        exp_grad_red_pred_test = exp_grad_red.predict(test)
        metric_test = ClassificationMetric(test, exp_grad_red_pred_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs_tets = {}
        metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
        metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
        metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
        metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
        metric_arrs_tets['disp_imp']=(logsumexp(metric_test.disparate_impact()))

        a_test=metric_test.accuracy()
        EGRlistdf_test.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc'],metric_arrs['disp_imp']])



A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.06968186201801335, 'eq_opp_diff': -0.01325444092092709, 'avg_odds_diff': -0.008387585621238931, 'bal_acc': 0.6661826597668907, 'disp_imp': 0.6437095386626551}
0.7920734159628533  !accuracy!
6838-34188 0-6837
alpha =  0


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.06908012659635496, 'eq_opp_diff': -0.011567756501626059, 'avg_odds_diff': -0.0076228412265691575, 'bal_acc': 0.6649078463810152, 'disp_imp': 0.6454960853734681}
0.791415304742057  !accuracy!
6838-34188 0-6837
alpha =  0.001


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

...

{'stat_par_diff': -0.06399401258505158, 'eq_opp_diff': -0.011880487967875608, 'avg_odds_diff': -0.004392627516117589, 'bal_acc': 0.6661339729535872, 'disp_imp': 0.6717001007444416}
0.791167007897046  !accuracy!
0-27351 27352-34188
alpha =  0


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.06485372593496055, 'eq_opp_diff': -0.0122522242432343, 'avg_odds_diff': -0.003962134554038793, 'bal_acc': 0.6707765527541029, 'disp_imp': 0.6814398947883822}
0.7901798771570635  !accuracy!
0-27351 27352-34188
alpha =  0.001


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.06436076197579299, 'eq_opp_diff': -0.013187466948704374, 'avg_odds_diff': -0.004171221921125325, 'bal_acc': 0.670261324649474, 'disp_imp': 0.6829224663784367}
0.790033635565955  !accuracy!
0-27351 27352-34188
alpha =  0.003


In [ ]:
import pandas as pd
dfEGR = pd.DataFrame (EGRlistdf, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfEGR

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.000,79.207342,-0.069682,-0.013254,-0.008388,0.666183,0.643710
1,1,gini,10,0.001,79.141530,-0.069080,-0.011568,-0.007623,0.664908,0.645496
2,1,gini,10,0.003,78.955066,-0.066363,-0.010262,-0.005128,0.666107,0.666837
3,1,gini,100,0.000,79.192717,-0.069900,-0.012595,-0.008232,0.666245,0.643394
4,1,gini,100,0.001,79.119593,-0.071250,-0.012914,-0.008613,0.668662,0.645904
5,1,gini,100,0.003,79.119593,-0.066257,-0.017240,-0.009485,0.659760,0.646062
6,1,entropy,10,0.000,79.192717,-0.069463,-0.012533,-0.008027,0.665876,0.644431
7,1,entropy,10,0.001,79.137874,-0.068912,-0.013372,-0.008150,0.665411,0.647648
8,1,entropy,10,0.003,79.079376,-0.071595,-0.014879,-0.010323,0.665817,0.638682
9,1,entropy,100,0.000,79.167124,-0.068859,-0.010249,-0.006823,0.665392,0.646827


In [ ]:
import pandas as pd
dfEGR_test = pd.DataFrame (EGRlistdf_test, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfEGR_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.000,79.253395,-0.074685,0.001009,-0.002895,0.674127,0.643710
1,1,gini,10,0.001,79.355763,-0.076828,-0.003982,-0.005770,0.676259,0.645496
2,1,gini,10,0.003,79.198799,-0.074371,-0.000163,-0.002545,0.676783,0.666837
3,1,gini,100,0.000,79.191974,-0.075494,0.000677,-0.003603,0.673820,0.643394
4,1,gini,100,0.001,79.191974,-0.078602,-0.002116,-0.005984,0.676640,0.645904
5,1,gini,100,0.003,79.198799,-0.072386,-0.003836,-0.005041,0.667834,0.646062
6,1,entropy,10,0.000,79.151027,-0.075397,0.002341,-0.003034,0.673064,0.644431
7,1,entropy,10,0.001,79.314816,-0.076306,-0.001364,-0.004447,0.675892,0.647648
8,1,entropy,10,0.003,79.185150,-0.075983,0.004872,-0.001975,0.674553,0.638682
9,1,entropy,100,0.000,79.246571,-0.075803,-0.000655,-0.004166,0.674568,0.646827


In [ ]:
#best fairness
column = dfEGR["eq_opp_diff"].abs()
max_value_bf = column.min()
max_index_bf = column.idxmin()
print("best value for Equal opportunity difference is %s and its index is %s" % (round(dfEGR["eq_opp_diff"][max_index_bf],6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % dfEGR['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % dfEGR['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is -0.007038 and its index is 20
best Equal opportunity difference's fold is 2
cpp_alpha value for the best Equal opportunity difference is 0.003 


In [ ]:
#best accuracy
column = dfEGR["accuracy"]
max_value_a = column.max()
max_index_a = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_a,6),max_index_a))
print("best accuracy's fold is %s" % dfEGR['kfold'][max_index_a])
print("cpp_alpha value for the best accuracy is %s " % dfEGR['cpp_alpha'][max_index_a])


maximum value for accuracy is 79.207342 and its index is 0
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0 


In [ ]:
#best fairness adn accuracy models are applied to test set and their values are shown
columna = dfEGR_test["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  dfEGR_test["eq_opp_diff"].abs()
max_valuef = columnf.min()
max_indexf = columnf.idxmin()


print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (dfEGR_test["accuracy"][max_index_bf],dfEGR_test["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (dfEGR_test["accuracy"][max_index_a],dfEGR_test["eq_opp_diff"][max_index_a]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,dfEGR_test['kfold'][max_indexa],dfEGR_test['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(dfEGR_test["eq_opp_diff"][max_indexf],dfEGR_test['kfold'][max_indexf],dfEGR_test['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 78.72790554835187 and its Equal opportunity difference is 0.005258163477537436


 the accuracy of the test set when the best accuracy model is applied to, is 79.25339520917218 and its Equal opportunity difference is 0.0010093853993344282


 the best accuracy value revealed for the test set is 79.3557633249164 and get from fold 1 with cpp_alpha value 0.001


 the best Equal opportunity difference value revealed for the test set is 0.00016313955906821498 and get from fold 1 with cpp_alpha value 0.003


# German

In [ ]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/german

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/german


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc

--2022-05-20 09:19:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79793 (78K) [application/x-httpd-php]
Saving to: ‘german.data’

german.data         100%[===================>]  77.92K  --.-KB/s    in 0.02s   

2022-05-20 09:19:53 (3.77 MB/s) - ‘german.data’ saved [79793/79793]

--2022-05-20 09:19:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4679 (4.6K) [application/x-httpd-php]
Saving to: ‘german.doc’

german.doc          100%[===================>]   4.57K  --.-KB/s    in 0s 

In [ ]:
##German
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult,load_preproc_data_german
from aif360.datasets import GermanDataset

#STEP 2: We define where's the bias in the features of our dataset.

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig_german = load_preproc_data_german(['sex'])


#STEP 3: We split between training and test set.
dataset_orig_train, dataset_orig_test = dataset_orig_german.split([0.7], shuffle=True)
print("training data size", dataset_orig_train.features.shape)
print("dataset feature names", dataset_orig_train.feature_names)


#Normalize the dataset, both train and test. This should always be done in any machine learning pipeline!
scale_orig = StandardScaler()
X_train_german = scale_orig.fit_transform(dataset_orig_train.features)
y_train_german = dataset_orig_train.labels.ravel()

X_test_german = scale_orig.transform(dataset_orig_test.features) 
y_test_german = dataset_orig_test.labels.ravel()

training data size (700, 11)
dataset feature names ['age', 'sex', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']


In [ ]:
#q1  German
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
Cs=[0,0.0005,0.001]
n_estimat=[10,100]
criter=["gini","entropy"]
listdfgerman=[]
listdfgerman_test=[]
k=0
for t,v in kf.split(X_train_german):
  k+=1
  trainx, valx = X_train_german[t],X_train_german[v]
  trainy,valy = y_train_german[t],y_train_german[v]
  trainsbgerman = dataset_orig_train.subset(indexes=t)
  for crite in criter:
    for n_estim in n_estimat:
      for c in Cs:
        print("k= ",k)
        print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
        print("alpha = ",c)
        classifier = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
        classifier.fit(trainx,trainy)
        predictions = classifier.predict(trainx)

        test_pred = trainsbgerman.copy()
        predictions.resize((len(predictions),1))
        test_pred.labels = predictions
        metric = ClassificationMetric(trainsbgerman, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs = {}
        metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
        metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
        metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
        metric_arrs['bal_acc']=((metric.true_positive_rate() + metric.true_negative_rate()) / 2)
        metric_arrs['disp_imp']=(metric.disparate_impact())
        print(metric_arrs)
        cm = confusion_matrix(trainy, predictions)
        print(cm," cm")
        print(metric.accuracy()," !accuracy!")
        a=metric.accuracy()
        listdfgerman.append([k,crite,n_estim,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc'],metric_arrs['disp_imp']])

        #the values get when the current classifier applied to the Test set
        predicted_test_values=classifier.predict(X_test_german)
        test_pred_for_test = dataset_orig_test.copy()
        predicted_test_values.resize((len(predicted_test_values),1))
        test_pred_for_test.labels = predicted_test_values
        metric_test = ClassificationMetric(dataset_orig_test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        metric_arrs_tets = {}
        metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
        metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
        metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
        metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
        metric_arrs_tets['disp_imp']=(metric_test.disparate_impact())
        a_test=metric_test.accuracy()
        listdfgerman_test.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc'],metric_arrs_tets['disp_imp']])
      

k=  1
140-699 0-139
alpha =  0
{'stat_par_diff': -0.22280701754385968, 'eq_opp_diff': -0.13392341150348264, 'avg_odds_diff': -0.24517238396241955, 'bal_acc': 0.611142130405112, 'disp_imp': 0.7628384687208216}
[[371  27]
 [115  47]]  cm
0.7464285714285714  !accuracy!
k=  1
140-699 0-139
alpha =  0.0005
{'stat_par_diff': -0.22280701754385968, 'eq_opp_diff': -0.13392341150348264, 'avg_odds_diff': -0.24517238396241955, 'bal_acc': 0.611142130405112, 'disp_imp': 0.7628384687208216}
[[371  27]
 [115  47]]  cm
0.7464285714285714  !accuracy!
k=  1
140-699 0-139
alpha =  0.001
{'stat_par_diff': -0.24444444444444435, 'eq_opp_diff': -0.14958785777291106, 'avg_odds_diff': -0.2710421251346518, 'bal_acc': 0.6093119920590607, 'disp_imp': 0.7426900584795323}
[[372  26]
 [116  46]]  cm
0.7464285714285714  !accuracy!
k=  1
140-699 0-139
alpha =  0
{'stat_par_diff': -0.23333333333333328, 'eq_opp_diff': -0.14104084922590254, 'avg_odds_diff': -0.2588321129246396, 'bal_acc': 0.6074818537130096, 'disp_imp': 0

In [ ]:
import pandas as pd
dfGerman_q1 = pd.DataFrame (listdfgerman, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q1

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,74.642857,-0.222807,-0.133923,-0.245172,0.611142,0.762838
1,1,gini,10,0.0005,74.642857,-0.222807,-0.133923,-0.245172,0.611142,0.762838
2,1,gini,10,0.0010,74.642857,-0.244444,-0.149588,-0.271042,0.609312,0.742690
3,1,gini,100,0.0000,74.642857,-0.233333,-0.141041,-0.258832,0.607482,0.754386
4,1,gini,100,0.0005,74.642857,-0.233333,-0.141041,-0.258832,0.607482,0.754386
5,1,gini,100,0.0010,74.642857,-0.233333,-0.141041,-0.258832,0.607482,0.754386
6,1,entropy,10,0.0000,74.642857,-0.222807,-0.133923,-0.245172,0.611142,0.762838
7,1,entropy,10,0.0005,74.642857,-0.222807,-0.133923,-0.245172,0.611142,0.762838
8,1,entropy,10,0.0010,74.642857,-0.222807,-0.133923,-0.245172,0.611142,0.762838
9,1,entropy,100,0.0000,74.642857,-0.233333,-0.141041,-0.258832,0.607482,0.754386


In [ ]:
import pandas as pd
dfGerman_q1_test = pd.DataFrame (listdfgerman_test, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q1_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
1,1,gini,10,0.0005,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
2,1,gini,10,0.0010,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
3,1,gini,100,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
4,1,gini,100,0.0005,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
5,1,gini,100,0.0010,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
6,1,entropy,10,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
7,1,entropy,10,0.0005,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
8,1,entropy,10,0.0010,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781
9,1,entropy,100,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.735781


In [ ]:
#best fairness
column = dfGerman_q1["eq_opp_diff"].abs()
max_value_bf = column.min()
max_index_bf = column.idxmin()
print("best value for Equal opportunity difference is %s and its index is %s" % (round(dfGerman_q1["eq_opp_diff"][max_index_bf],6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % dfGerman_q1['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % dfGerman_q1['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is -0.073194 and its index is 17
best Equal opportunity difference's fold is 2
cpp_alpha value for the best Equal opportunity difference is 0.001 


In [ ]:
#best accuracy
column = dfGerman_q1["accuracy"]
max_value_a = column.max()
max_index_a = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_a,6),max_index_a))
print("best accuracy's fold is %s" % dfGerman_q1['kfold'][max_index_a])
print("cpp_alpha value for the best accuracy is %s " % dfGerman_q1['cpp_alpha'][max_index_a])


maximum value for accuracy is 74.642857 and its index is 0
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0 


In [ ]:
#best fairness and accuracy models are applied to test set and their values are shown
columna = dfGerman_q1_test["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  dfGerman_q1_test["eq_opp_diff"].abs()
max_indexf = columnf.idxmin()
max_valuef = dfGerman_q1_test["eq_opp_diff"][max_indexf]

print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q1_test["accuracy"][max_index_bf],dfGerman_q1_test["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q1_test["accuracy"][max_index_a],dfGerman_q1_test["eq_opp_diff"][max_index_a]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,dfGerman_q1_test['kfold'][max_indexa],dfGerman_q1_test['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuef,dfGerman_q1_test['kfold'][max_indexf],dfGerman_q1_test['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 72.33333333333334 and its Equal opportunity difference is -0.044791666666666674


 the accuracy of the test set when the best accuracy model is applied to, is 70.33333333333334 and its Equal opportunity difference is -0.20770833333333338


 the best accuracy value revealed for the test set is 72.33333333333334 and get from fold 2 with cpp_alpha value 0.001


 the best Equal opportunity difference value revealed for the test set is -0.044791666666666674 and get from fold 2 with cpp_alpha value 0.001


In [ ]:
#German q2 RW
from sklearn.model_selection import KFold
from scipy.special import logsumexp
kf = KFold(n_splits=5)
Cs=[0,0.0005,0.001]
n_estimat=[10,100]
criter=["gini","entropy"]
RW_model = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
k=0
RWlistdf_german=[]
listdf_germantest_RW=[]
for t,v in kf.split(X_train_german):
  k+=1
  trainx, valx = X_train_german[t],X_train_german[v]
  trainy,valy = y_train_german[t],y_train_german[v]
  trainsbgerman = dataset_orig_train.subset(indexes=t)
  for crite in criter:
   for n_estim in n_estimat:
    for c in Cs:
      trainsbgerman = RW_model.fit_transform(trainsbgerman)
      learner = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
      learner.fit(trainx,trainy,sample_weight=trainsbgerman.instance_weights)

      predictions = learner.predict(trainx)
      print("Accuracy", sum(predictions==trainy)/len(trainy))
      
    
      test_pred = trainsbgerman.copy()
      predictions.resize((len(predictions),1))
      test_pred.labels = predictions

      metric = ClassificationMetric(trainsbgerman, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs = {}
      metric_arrs['stat_par_diff']=(logsumexp(metric.statistical_parity_difference()))
      metric_arrs['eq_opp_diff']=(logsumexp(metric.equal_opportunity_difference()))
      metric_arrs['avg_odds_diff']=(logsumexp(metric.average_odds_difference()))
      metric_arrs['bal_acc']=((logsumexp(metric.true_positive_rate())+ logsumexp(metric.true_negative_rate())) / 2)
      metric_arrs['disp_imp']=(logsumexp(metric.disparate_impact()))
      print(metric_arrs)

      print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
      print("alpha = ",c)
      a=metric.accuracy()
      RWlistdf_german.append([k,crite,n_estim,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc'],metric_arrs['disp_imp']])

      #the values get when the current classifier applied to the Test set
      predicted_test_values=learner.predict(X_test_german)
      test_pred_for_test = dataset_orig_test.copy()
      predicted_test_values.resize((len(predicted_test_values),1))
      test_pred_for_test.labels = predicted_test_values

      metric_test = ClassificationMetric(dataset_orig_test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs_tets = {}
      metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
      metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
      metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
      metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
      a_test=metric_test.accuracy()
      listdf_germantest_RW.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc'],metric_arrs['disp_imp']])



Accuracy 0.7464285714285714
{'stat_par_diff': -0.19828888842615333, 'eq_opp_diff': -0.13392341150348275, 'avg_odds_diff': -0.24517238396241942, 'bal_acc': 0.5972812986031084, 'disp_imp': 0.7881381781296969}
140-699 0-139
alpha =  0
Accuracy 0.7446428571428572
{'stat_par_diff': -0.1403654626455847, 'eq_opp_diff': -0.09118836876843983, 'avg_odds_diff': -0.17618581497585045, 'bal_acc': 0.5888433080222608, 'disp_imp': 0.8500264796489765}
140-699 0-139
alpha =  0.0005
Accuracy 0.7446428571428572
{'stat_par_diff': -0.16169809826393466, 'eq_opp_diff': -0.10828238586245686, 'avg_odds_diff': -0.20060583939587512, 'bal_acc': 0.5911980960913346, 'disp_imp': 0.8272336187717412}
140-699 0-139
alpha =  0.001
Accuracy 0.7464285714285714
{'stat_par_diff': -0.17695625280780347, 'eq_opp_diff': -0.11682939440946549, 'avg_odds_diff': -0.22075235954239508, 'bal_acc': 0.5949265105340347, 'disp_imp': 0.8109310390069322}
140-699 0-139
alpha =  0
Accuracy 0.7464285714285714
{'stat_par_diff': -0.176956252807803

In [ ]:
import pandas as pd
dfGerman_q2_reweigh = pd.DataFrame (RWlistdf_german, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q2_reweigh

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,73.685934,-0.198289,-0.133923,-0.245172,0.597281,0.788138
1,1,gini,10,0.0005,73.776622,-0.140365,-0.091188,-0.176186,0.588843,0.850026
2,1,gini,10,0.0010,73.681309,-0.161698,-0.108282,-0.200606,0.591198,0.827234
3,1,gini,100,0.0000,73.781247,-0.176956,-0.116829,-0.220752,0.594927,0.810931
4,1,gini,100,0.0005,73.781247,-0.176956,-0.116829,-0.220752,0.594927,0.810931
5,1,gini,100,0.0010,73.723307,-0.151268,-0.098306,-0.189846,0.584404,0.840239
6,1,entropy,10,0.0000,73.685934,-0.198289,-0.133923,-0.245172,0.597281,0.788138
7,1,entropy,10,0.0005,73.685934,-0.198289,-0.133923,-0.245172,0.597281,0.788138
8,1,entropy,10,0.0010,73.685934,-0.198289,-0.133923,-0.245172,0.597281,0.788138
9,1,entropy,100,0.0000,73.781247,-0.176956,-0.116829,-0.220752,0.594927,0.810931


In [ ]:
import pandas as pd
dfGerman_q2_reweigh_test = pd.DataFrame (listdf_germantest_RW, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q2_reweigh_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.788138
1,1,gini,10,0.0005,70.666667,-0.218911,-0.176458,-0.256240,0.547490,0.850026
2,1,gini,10,0.0010,70.666667,-0.241638,-0.192083,-0.284886,0.550967,0.827234
3,1,gini,100,0.0000,70.333333,-0.230274,-0.192083,-0.264052,0.545153,0.810931
4,1,gini,100,0.0005,70.333333,-0.230274,-0.192083,-0.264052,0.545153,0.810931
5,1,gini,100,0.0010,70.666667,-0.218911,-0.176458,-0.256240,0.547490,0.840239
6,1,entropy,10,0.0000,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.788138
7,1,entropy,10,0.0005,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.788138
8,1,entropy,10,0.0010,70.333333,-0.253002,-0.207708,-0.292698,0.548631,0.788138
9,1,entropy,100,0.0000,70.333333,-0.230274,-0.192083,-0.264052,0.545153,0.810931


In [ ]:
#best fairness
column = dfGerman_q2_reweigh["eq_opp_diff"].abs()
max_index_bf = column.idxmin()
max_value_bf = dfGerman_q2_reweigh["eq_opp_diff"][max_index_bf]

print("best value for Equal opportunity difference is %s and its index is %s" % (round(max_value_bf,6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % dfGerman_q2_reweigh['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % dfGerman_q2_reweigh['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is -0.005844 and its index is 49
best Equal opportunity difference's fold is 5
cpp_alpha value for the best Equal opportunity difference is 0.0005 


In [ ]:
#best accuracy
column = dfGerman_q2_reweigh["accuracy"]
max_value_a = column.max()
max_index_a = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_a,6),max_index_a))
print("best accuracy's fold is %s" % dfGerman_q2_reweigh['kfold'][max_index_a])
print("cpp_alpha value for the best accuracy is %s " % dfGerman_q2_reweigh['cpp_alpha'][max_index_a])


maximum value for accuracy is 73.781247 and its index is 4
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0005 


In [ ]:
#best fairness and accuracy models are applied to test set and their values are shown
columna = dfGerman_q2_reweigh_test["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  dfGerman_q2_reweigh_test["eq_opp_diff"].abs()
max_indexf = columnf.idxmin()
max_valuef = dfGerman_q2_reweigh_test["eq_opp_diff"][max_indexf]

print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q2_reweigh_test["accuracy"][max_index_bf],dfGerman_q2_reweigh_test["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q2_reweigh_test["accuracy"][max_index_a],dfGerman_q2_reweigh_test["eq_opp_diff"][max_index_a]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,dfGerman_q2_reweigh_test['kfold'][max_indexa],dfGerman_q2_reweigh_test['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuef,dfGerman_q2_reweigh_test['kfold'][max_indexf],dfGerman_q2_reweigh_test['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 71.66666666666667 and its Equal opportunity difference is 0.026666666666666616


 the accuracy of the test set when the best accuracy model is applied to, is 70.33333333333334 and its Equal opportunity difference is -0.19208333333333338


 the best accuracy value revealed for the test set is 73.0 and get from fold 3 with cpp_alpha value 0.0


 the best Equal opportunity difference value revealed for the test set is 0.011041666666666616 and get from fold 5 with cpp_alpha value 0.001


In [ ]:
#EGR q2
from sklearn.model_selection import KFold
from scipy.special import logsumexp
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction
kf = KFold(n_splits=5)
Cs=[0,0.0005,0.001]
n_estimat=[10,100]
criter=["gini","entropy"]
EGRlistdf_geramn=[]
EGRlistdf_test_german=[]
k=0
for t,v in kf.split(X_train_german):
  k+=1
  trainx, valx = X_train_german[t],X_train_german[v]
  trainy,valy = y_train_german[t],y_train_german[v]
  trainsbgerman = dataset_orig_train.subset(indexes=t)
  for crite in criter:
   for n_estim in n_estimat:
    for c in Cs:
    
      learner = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
      exp_grad_red = ExponentiatedGradientReduction(estimator=learner, 
                                                constraints="EqualizedOdds",
                                                drop_prot_attr=False)
      exp_grad_red.fit(trainsbgerman)

      exp_grad_red_pred = exp_grad_red.predict(trainsbgerman)

      metric = ClassificationMetric(trainsbgerman, 
                                    exp_grad_red_pred,
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)
      
    
      metric_arrs = {}
      metric_arrs['stat_par_diff']=(logsumexp(metric.statistical_parity_difference()))
      metric_arrs['eq_opp_diff']=(logsumexp(metric.equal_opportunity_difference()))
      metric_arrs['avg_odds_diff']=(logsumexp(metric.average_odds_difference()))
      metric_arrs['bal_acc']=((logsumexp(metric.true_positive_rate())+ logsumexp(metric.true_negative_rate())) / 2)
      metric_arrs['disp_imp']=(logsumexp(metric.disparate_impact()))
      print(metric_arrs)
      print(metric.accuracy()," !accuracy!")

      print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
      print("alpha = ",c)
      a=metric.accuracy()
      EGRlistdf_geramn.append([k,crite,n_estim,c,a*100 ,metric_arrs['stat_par_diff'],metric_arrs['eq_opp_diff'],metric_arrs['avg_odds_diff'],metric_arrs['bal_acc'],metric_arrs['disp_imp']])


      #the values get when the current classifier applied to the Test set
      exp_grad_red_test = ExponentiatedGradientReduction(estimator=learner, 
                                                constraints="EqualizedOdds",
                                                drop_prot_attr=False)
      exp_grad_red_test.fit(trainsbgerman)
      exp_grad_red_pred_test = exp_grad_red.predict(dataset_orig_test)
      metric_test = ClassificationMetric(dataset_orig_test, exp_grad_red_pred_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs_tets = {}
      metric_arrs_tets['stat_par_diff']=(metric_test.statistical_parity_difference())
      metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
      metric_arrs_tets['avg_odds_diff']=(metric_test.average_odds_difference())
      metric_arrs_tets['bal_acc']=((metric_test.true_positive_rate() + metric_test.true_negative_rate()) / 2)
      metric_arrs_tets['disp_imp']=(logsumexp(metric_test.disparate_impact()))

      a_test=metric_test.accuracy()
      EGRlistdf_test_german.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['stat_par_diff'],metric_arrs_tets['eq_opp_diff'],metric_arrs_tets['avg_odds_diff'],metric_arrs_tets['bal_acc'],metric_arrs['disp_imp']])


      



    



  

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.0061403508771930015, 'eq_opp_diff': -0.005718283298354443, 'avg_odds_diff': 0.009406370616335036, 'bal_acc': 0.5559122774365656, 'disp_imp': 0.9934640522875817}
0.7303571428571428  !accuracy!
140-699 0-139
alpha =  0


Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Seri

{'stat_par_diff': -0.03391812865497079, 'eq_opp_diff': -0.031359308939380104, 'avg_odds_diff': -0.019287158077193667, 'bal_acc': 0.5583162727216329, 'disp_imp': 0.9638966697790227}
0.7285714285714285  !accuracy!
140-699 0-139
alpha =  0.0005


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

...

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.016909867490779718, 'eq_opp_diff': -0.018831168831168865, 'avg_odds_diff': 0.0029617096518504415, 'bal_acc': 0.5500754147812972, 'disp_imp': 0.9818959328276681}
0.7107142857142857  !accuracy!
0-559 560-699
alpha =  0.0005


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

{'stat_par_diff': -0.06086094549483234, 'eq_opp_diff': -0.013311688311688274, 'avg_odds_diff': -0.055667083307928356, 'bal_acc': 0.5732277526395173, 'disp_imp': 0.9352070271276925}
0.7267857142857143  !accuracy!
0-559 560-699
alpha =  0.001


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a fut

In [ ]:
import pandas as pd
dfGerman_q2_EGR = pd.DataFrame (EGRlistdf_geramn, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q2_EGR

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,73.035714,-0.006140,-0.005718,0.009406,0.555912,0.993464
1,1,gini,10,0.0005,72.857143,-0.033918,-0.031359,-0.019287,0.558316,0.963897
2,1,gini,10,0.0010,72.857143,-0.000585,-0.005718,0.017343,0.552826,0.999378
3,1,gini,100,0.0000,73.571429,-0.000585,0.011376,0.010017,0.561511,0.999378
4,1,gini,100,0.0005,73.928571,-0.045029,-0.014265,-0.042486,0.573175,0.952070
5,1,gini,100,0.0010,72.857143,-0.011696,-0.014265,0.005133,0.554656,0.987551
6,1,entropy,10,0.0000,73.214286,-0.022807,-0.014265,-0.010740,0.560829,0.975724
7,1,entropy,10,0.0005,72.678571,-0.028363,-0.031359,-0.011351,0.555230,0.969810
8,1,entropy,10,0.0010,73.392857,-0.017251,-0.005718,-0.006467,0.562085,0.981637
9,1,entropy,100,0.0000,73.214286,-0.011696,-0.005718,0.001470,0.558999,0.987551


In [ ]:
import pandas as pd
dfGerman_q2_EGR_test = pd.DataFrame (EGRlistdf_test_german, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','stat_par_diff','eq_opp_diff','avg_odds_diff','bal_acc','disp_imp'])
dfGerman_q2_EGR_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,stat_par_diff,eq_opp_diff,avg_odds_diff,bal_acc,disp_imp
0,1,gini,10,0.0000,69.000000,-0.123285,-0.138542,-0.112282,0.514942,0.993464
1,1,gini,10,0.0005,70.000000,-0.082547,-0.098333,-0.071344,0.518474,0.963897
2,1,gini,10,0.0010,71.333333,-0.037093,-0.035833,-0.040094,0.527820,0.999378
3,1,gini,100,0.0000,70.000000,-0.014365,-0.051458,0.014593,0.508042,0.999378
4,1,gini,100,0.0005,71.333333,-0.037093,-0.035833,-0.040094,0.527820,0.952070
5,1,gini,100,0.0010,70.333333,-0.003002,-0.035833,0.022406,0.510378,0.987551
6,1,entropy,10,0.0000,70.333333,-0.116638,-0.113958,-0.120823,0.527766,0.975724
7,1,entropy,10,0.0005,70.333333,-0.123285,-0.107292,-0.138323,0.531243,0.969810
8,1,entropy,10,0.0010,69.666667,-0.100557,-0.107292,-0.096657,0.519615,0.981637
9,1,entropy,100,0.0000,71.000000,-0.048456,-0.051458,-0.047907,0.525484,0.987551


In [ ]:
#best fairness
column = dfGerman_q2_EGR["eq_opp_diff"].abs()
max_value_bf = column.min()
max_index_bf = column.idxmin()
print("best value for Equal opportunity difference is %s and its index is %s" % (round(dfGerman_q2_EGR["eq_opp_diff"][max_index_bf],6),max_index_bf))
print("best Equal opportunity difference's fold is %s" % dfGerman_q2_EGR['kfold'][max_index_bf])
print("cpp_alpha value for the best Equal opportunity difference is %s " % dfGerman_q2_EGR['cpp_alpha'][max_index_bf])

best value for Equal opportunity difference is 3.4e-05 and its index is 24
best Equal opportunity difference's fold is 3
cpp_alpha value for the best Equal opportunity difference is 0.0 


In [ ]:
#best accuracy
column = dfGerman_q2_EGR["accuracy"]
max_value_a = column.max()
max_index_a = column.idxmax()
print("maximum value for accuracy is %s and its index is %s" % (round(max_value_a,6),max_index_a))
print("best accuracy's fold is %s" % dfGerman_q2_EGR['kfold'][max_index_a])
print("cpp_alpha value for the best accuracy is %s " % dfGerman_q2_EGR['cpp_alpha'][max_index_a])


maximum value for accuracy is 73.928571 and its index is 4
best accuracy's fold is 1
cpp_alpha value for the best accuracy is 0.0005 


In [ ]:
#best fairness and accuracy models are applied to test set and their values are shown
columna = dfGerman_q2_EGR_test["accuracy"]
max_valuea = columna.max()
max_indexa = columna.idxmax()

columnf =  dfGerman_q2_EGR_test["eq_opp_diff"].abs()
max_indexf = columnf.idxmin()
max_valuef = dfGerman_q2_EGR_test["eq_opp_diff"][max_indexf]

print(" the accuracy of the test set when the best fairness model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q2_EGR_test["accuracy"][max_index_bf],dfGerman_q2_EGR_test["eq_opp_diff"][max_index_bf]))
print("\n")
print(" the accuracy of the test set when the best accuracy model is applied to, is %s and its Equal opportunity difference is %s" % (dfGerman_q2_EGR_test["accuracy"][max_index_a],dfGerman_q2_EGR_test["eq_opp_diff"][max_index_a]))
print("\n")
print(" the best accuracy value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuea,dfGerman_q2_EGR_test['kfold'][max_indexa],dfGerman_q2_EGR_test['cpp_alpha'][max_indexa]))
print("\n")
print(" the best Equal opportunity difference value revealed for the test set is %s and get from fold %s with cpp_alpha value %s" %(max_valuef,dfGerman_q2_EGR_test['kfold'][max_indexf],dfGerman_q2_EGR_test['cpp_alpha'][max_indexf]))


 the accuracy of the test set when the best fairness model is applied to, is 71.66666666666667 and its Equal opportunity difference is -0.004583333333333384


 the accuracy of the test set when the best accuracy model is applied to, is 71.33333333333334 and its Equal opportunity difference is -0.035833333333333384


 the best accuracy value revealed for the test set is 73.33333333333333 and get from fold 5 with cpp_alpha value 0.0005


 the best Equal opportunity difference value revealed for the test set is -0.0002083333333333659 and get from fold 2 with cpp_alpha value 0.001


# q3

In [ ]:
from math import log
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
 


#calculate mymodel selection 
def calculate_mymodel(metric):
  mm= metric.accuracy()*100 - abs(metric.equal_opportunity_difference()) 
  return mm

#Receiver Operating Characteristics
def calculate_roc(testy,probs):
	auc = roc_auc_score(testy, probs)
	return auc




In [ ]:
#q3
#German
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
Cs=[0,0.0005,0.001]
n_estimat=[10,100]
criter=["gini","entropy"]
listdfgerman_q3=[]
listdfgerman_test_q3=[]
k=0
for t,v in kf.split(X_train_german):
  k+=1
  trainx, valx = X_train_german[t],X_train_german[v]
  trainy,valy = y_train_german[t],y_train_german[v]
  trainsbgerman = dataset_orig_train.subset(indexes=t)
  for crite in criter:
   for n_estim in n_estimat:
    for c in Cs:
      print("k= ",k)
      print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
      print("alpha = ",c)
      classifier = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
      classifier.fit(trainx,trainy)
      predictions = classifier.predict(trainx)
      p=classifier.get_params(deep=True)
      print(p,"parameters")


      test_pred = trainsbgerman.copy()
      predictions.resize((len(predictions),1))
      test_pred.labels = predictions

      metric = ClassificationMetric(trainsbgerman, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs = {}
      metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
      print(metric_arrs)
      print(metric.accuracy()," !accuracy!")
      cm = confusion_matrix(trainy, predictions)
      print(cm," cm")

      probs = predictions  
      auc = roc_auc_score(trainy,probs)
      a=metric.accuracy()
      cmm=calculate_mymodel(metric)
      listdfgerman_q3.append([k,crite,n_estim,c,a*100 ,metric_arrs['eq_opp_diff'],auc,cmm])



      #the values get when the current classifier applied to the Test set
      predicted_test_values=classifier.predict(X_test_german)
      test_pred_for_test = dataset_orig_test.copy()
      predicted_test_values.resize((len(predicted_test_values),1))
      test_pred_for_test.labels = predicted_test_values
      metric_test = ClassificationMetric(dataset_orig_test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs_tets = {}
      metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
      a_test=metric_test.accuracy()

      probs = predicted_test_values 
      auc = roc_auc_score(y_test_german,probs)
      cmm=calculate_mymodel(metric_test)
      listdfgerman_test_q3.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['eq_opp_diff'],auc,cmm])


  

k=  1
140-699 0-139
alpha =  0
{'bootstrap': True, 'ccp_alpha': 0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False} parameters
{'eq_opp_diff': -0.13392341150348264}
0.7464285714285714  !accuracy!
[[371  27]
 [115  47]]  cm
k=  1
140-699 0-139
alpha =  0.0005
{'bootstrap': True, 'ccp_alpha': 0.0005, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False} parameters
{'eq_opp_diff': -0.13392341150348264

In [ ]:
import pandas as pd
dfGerman_q3 = pd.DataFrame (listdfgerman_q3, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','eq_opp_diff','auc','cmm'])
dfGerman_q3

,kfold,criterion,number of trees,cpp_alpha,accuracy,eq_opp_diff,auc,cmm
0,1,gini,10,0.0000,74.642857,-0.133923,0.611142,74.508934
1,1,gini,10,0.0005,74.642857,-0.133923,0.611142,74.508934
2,1,gini,10,0.0010,74.642857,-0.149588,0.609312,74.493269
3,1,gini,100,0.0000,74.642857,-0.141041,0.607482,74.501816
4,1,gini,100,0.0005,74.642857,-0.141041,0.607482,74.501816
5,1,gini,100,0.0010,74.642857,-0.141041,0.607482,74.501816
6,1,entropy,10,0.0000,74.642857,-0.133923,0.611142,74.508934
7,1,entropy,10,0.0005,74.642857,-0.133923,0.611142,74.508934
8,1,entropy,10,0.0010,74.642857,-0.133923,0.611142,74.508934
9,1,entropy,100,0.0000,74.642857,-0.141041,0.607482,74.501816


In [ ]:
import pandas as pd
dfGerman_q3_test = pd.DataFrame (listdfgerman_test_q3, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','eq_opp_diff','auc','cmm'])
dfGerman_q3_test

,kfold,criterion,number of trees,cpp_alpha,accuracy,eq_opp_diff,auc,cmm
0,1,gini,10,0.0000,70.333333,-0.207708,0.548631,70.125625
1,1,gini,10,0.0005,70.333333,-0.207708,0.548631,70.125625
2,1,gini,10,0.0010,70.333333,-0.207708,0.548631,70.125625
3,1,gini,100,0.0000,70.333333,-0.207708,0.548631,70.125625
4,1,gini,100,0.0005,70.333333,-0.207708,0.548631,70.125625
5,1,gini,100,0.0010,70.333333,-0.207708,0.548631,70.125625
6,1,entropy,10,0.0000,70.333333,-0.207708,0.548631,70.125625
7,1,entropy,10,0.0005,70.333333,-0.207708,0.548631,70.125625
8,1,entropy,10,0.0010,70.333333,-0.207708,0.548631,70.125625
9,1,entropy,100,0.0000,70.333333,-0.207708,0.548631,70.125625


In [ ]:
#best cmm
column = dfGerman_q3["cmm"]
max_value_bf = column.max()
max_index_bf = column.idxmax()
print("best value for the created model choosing is %s and its index is %s" % (round(max_value_bf,6),max_index_bf))
print("this model's accuracy is %s and EOD is %s " % (dfGerman_q3["accuracy"][max_index_bf],dfGerman_q3["eq_opp_diff"][max_index_bf]))

print("When this model is applied to test set the accuracy obtained is %s and the EOD obtained is %s " %(dfGerman_q3_test['accuracy'][max_index_bf],dfGerman_q3_test['eq_opp_diff'][max_index_bf]))

best value for the created model choosing is 74.508934 and its index is 0
this model's accuracy is 74.64285714285714 and EOD is -0.13392341150348264 
When this model is applied to test set the accuracy obtained is 70.33333333333334 and the EOD obtained is -0.20770833333333338 


In [ ]:
#best roc
column = dfGerman_q3["auc"]
max_value_bf = column.max()
max_index_bf = column.idxmax()
print("best value for the ROC  is %s and its index is %s" % (round(max_value_bf,6),max_index_bf))
print("this model's accuracy is %s and EOD is %s " % (dfGerman_q3["accuracy"][max_index_bf],dfGerman_q3["eq_opp_diff"][max_index_bf]))
print("When this model is applied to test set the accuracy obtained is %s and the EOD obtained is %s " %(dfGerman_q3_test['accuracy'][max_index_bf],dfGerman_q3_test['eq_opp_diff'][max_index_bf]))


best value for the ROC  is 0.62663 and its index is 27
this model's accuracy is 72.67857142857143 and EOD is -0.315070842910563 
When this model is applied to test set the accuracy obtained is 68.33333333333333 and the EOD obtained is -0.38625 


In [ ]:
#German q3 RW
from sklearn.model_selection import KFold
from scipy.special import logsumexp
kf = KFold(n_splits=5)
Cs=[0,0.0005,0.001]
n_estimat=[10,100]
criter=["gini","entropy"]
RW_model = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
k=0
RWlistdf_german_q3=[]
listdf_germantest_RW_q3=[]
for t,v in kf.split(X_train_german):
  k+=1
  trainx, valx = X_train_german[t],X_train_german[v]
  trainy,valy = y_train_german[t],y_train_german[v]
  trainsbgerman = dataset_orig_train.subset(indexes=t)
  for crite in criter:
   for n_estim in n_estimat:
    for c in Cs:
      trainsbgerman = RW_model.fit_transform(trainsbgerman)
      learner = RandomForestClassifier(n_estimators = n_estim, criterion = crite, random_state = 0,ccp_alpha=c)
      learner.fit(trainx,trainy,sample_weight=trainsbgerman.instance_weights)

      predictions = learner.predict(trainx)
    
      test_pred = trainsbgerman.copy()
      predictions.resize((len(predictions),1))
      test_pred.labels = predictions

      metric = ClassificationMetric(trainsbgerman, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs = {}
     
      metric_arrs['eq_opp_diff']=(logsumexp(metric.equal_opportunity_difference()))
    
      print(metric_arrs)
      probs = predictions  
      auc = roc_auc_score(trainy,probs)
      a=metric.accuracy()
      cmm=calculate_mymodel(metric)
      RWlistdf_german_q3.append([k,crite,n_estim,c,a*100 ,metric_arrs['eq_opp_diff'],auc,cmm])

      print("%s-%s %s-%s" % (t[0],t[-1],v[0],v[-1]))
      print("alpha = ",c)
    

      #the values get when the current classifier applied to the Test set
      predicted_test_values=learner.predict(X_test_german)
      test_pred_for_test = dataset_orig_test.copy()
      predicted_test_values.resize((len(predicted_test_values),1))
      test_pred_for_test.labels = predicted_test_values

      metric_test = ClassificationMetric(dataset_orig_test, test_pred_for_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs_tets = {}
      metric_arrs_tets['eq_opp_diff']=(metric_test.equal_opportunity_difference())
      a_test=metric_test.accuracy()

      probs = predicted_test_values 
      auc = roc_auc_score(y_test_german,probs)
      cmm=calculate_mymodel(metric_test)

      listdf_germantest_RW_q3.append([k,crite,n_estim,c,a_test*100 ,metric_arrs_tets['eq_opp_diff'],auc,cmm])



{'eq_opp_diff': -0.13392341150348275}
140-699 0-139
alpha =  0
{'eq_opp_diff': -0.09118836876843983}
140-699 0-139
alpha =  0.0005
{'eq_opp_diff': -0.10828238586245686}
140-699 0-139
alpha =  0.001
{'eq_opp_diff': -0.11682939440946549}
140-699 0-139
alpha =  0
{'eq_opp_diff': -0.1168293944094656}
140-699 0-139
alpha =  0.0005
{'eq_opp_diff': -0.09830580649085974}
140-699 0-139
alpha =  0.001
{'eq_opp_diff': -0.13392341150348264}
140-699 0-139
alpha =  0
{'eq_opp_diff': -0.13392341150348264}
140-699 0-139
alpha =  0.0005
{'eq_opp_diff': -0.13392341150348264}
140-699 0-139
alpha =  0.001
{'eq_opp_diff': -0.11682939440946571}
140-699 0-139
alpha =  0
{'eq_opp_diff': -0.11682939440946571}
140-699 0-139
alpha =  0.0005
{'eq_opp_diff': -0.11682939440946549}
140-699 0-139
alpha =  0.001
{'eq_opp_diff': -0.07581078289609577}
0-699 140-279
alpha =  0
{'eq_opp_diff': -0.07581078289609577}
0-699 140-279
alpha =  0.0005
{'eq_opp_diff': -0.06610204503201811}
0-699 140-279
alpha =  0.001
{'eq_opp_di

In [ ]:
import pandas as pd
dfGerman_q3_RW = pd.DataFrame (RWlistdf_german_q3, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','eq_opp_diff','auc','cmm'])
dfGerman_q3_RW

,kfold,criterion,number of trees,cpp_alpha,accuracy,eq_opp_diff,auc,cmm
0,1,gini,10,0.0000,73.685934,-0.133923,0.611142,73.552011
1,1,gini,10,0.0005,73.776622,-0.091188,0.598905,73.685433
2,1,gini,10,0.0010,73.681309,-0.108282,0.602565,73.573026
3,1,gini,100,0.0000,73.781247,-0.116829,0.607482,73.664418
4,1,gini,100,0.0005,73.781247,-0.116829,0.607482,73.664418
5,1,gini,100,0.0010,73.723307,-0.098306,0.595245,73.625001
6,1,entropy,10,0.0000,73.685934,-0.133923,0.611142,73.552011
7,1,entropy,10,0.0005,73.685934,-0.133923,0.611142,73.552011
8,1,entropy,10,0.0010,73.685934,-0.133923,0.611142,73.552011
9,1,entropy,100,0.0000,73.781247,-0.116829,0.607482,73.664418


In [ ]:
import pandas as pd
dfGerman_q3_RW_test = pd.DataFrame (listdf_germantest_RW_q3, columns = ['kfold','criterion','number of trees','cpp_alpha','accuracy','eq_opp_diff','auc','cmm'])
dfGerman_q3_RW_test


,kfold,criterion,number of trees,cpp_alpha,accuracy,eq_opp_diff,auc,cmm
0,1,gini,10,0.0000,70.333333,-0.207708,0.548631,70.125625
1,1,gini,10,0.0005,70.666667,-0.176458,0.547490,70.490208
2,1,gini,10,0.0010,70.666667,-0.192083,0.550967,70.474583
3,1,gini,100,0.0000,70.333333,-0.192083,0.545153,70.141250
4,1,gini,100,0.0005,70.333333,-0.192083,0.545153,70.141250
5,1,gini,100,0.0010,70.666667,-0.176458,0.547490,70.490208
6,1,entropy,10,0.0000,70.333333,-0.207708,0.548631,70.125625
7,1,entropy,10,0.0005,70.333333,-0.207708,0.548631,70.125625
8,1,entropy,10,0.0010,70.333333,-0.207708,0.548631,70.125625
9,1,entropy,100,0.0000,70.333333,-0.192083,0.545153,70.141250


In [ ]:
#best cmm
column = dfGerman_q3_RW["cmm"]
max_value_bf = column.max()
max_index_bf = column.idxmax()
print("best value for the created model choosing is %s and its index is %s" % (round(max_value_bf,6),max_index_bf))
print("this model's accuracy is %s and EOD is %s " % (dfGerman_q3_RW["accuracy"][max_index_bf],dfGerman_q3_RW["eq_opp_diff"][max_index_bf]))

print("When this model is applied to test set the accuracy obtained is %s and the EOD obtained is %s " %(dfGerman_q3_RW_test['accuracy'][max_index_bf],dfGerman_q3_RW_test['eq_opp_diff'][max_index_bf]))

best value for the created model choosing is 73.685433 and its index is 1
this model's accuracy is 73.77662169154834 and EOD is -0.09118836876843983 
When this model is applied to test set the accuracy obtained is 70.66666666666667 and the EOD obtained is -0.17645833333333338 


In [ ]:
#best roc
column = dfGerman_q3_RW["auc"]
max_value_bf = column.max()
max_index_bf = column.idxmax()
print("best value for the ROC  is %s and its index is %s" % (round(max_value_bf,6),max_index_bf))
print("this model's accuracy is %s and EOD is %s " % (dfGerman_q3_RW["accuracy"][max_index_bf],dfGerman_q3_RW["eq_opp_diff"][max_index_bf]))
print("When this model is applied to test set the accuracy obtained is %s and the EOD obtained is %s " %(dfGerman_q3_RW_test['accuracy'][max_index_bf],dfGerman_q3_RW_test['eq_opp_diff'][max_index_bf]))


best value for the ROC  is 0.615844 and its index is 43
this model's accuracy is 71.90688221718274 and EOD is -0.047665056360708835 
When this model is applied to test set the accuracy obtained is 70.0 and the EOD obtained is -0.12979166666666664 
